# Modeling Uneven Outdoor Terrain in NVIDIA Omniverse

This tutorial will guide you through the process of creating uneven outdoor terrain in NVIDIA Omniverse using Python scripting. We will utilize Omniverse Kit's API to generate and manipulate terrain within the Omniverse environment.

## Table of Contents

1. [Introduction](#introduction)
2. [Prerequisites](#prerequisites)
3. [Setting Up the Environment](#setting-up-the-environment)
4. [Generating Uneven Terrain](#generating-uneven-terrain)
5. [Applying Textures and Materials](#applying-textures-and-materials)
6. [Simulating the Terrain](#simulating-the-terrain)
7. [Conclusion](#conclusion)

---

## Introduction

NVIDIA Omniverse is a powerful platform for real-time simulation and collaboration. Modeling uneven terrain is essential for simulations involving outdoor environments, such as robotics, gaming, and virtual reality. In this tutorial, we'll create a terrain using procedural noise functions and manipulate it to simulate an uneven outdoor landscape.

## Prerequisites

Before you begin, ensure you have the following:

- **NVIDIA Omniverse Kit or Omniverse Code** installed.
- Basic understanding of Python programming.
- Familiarity with Omniverse's scripting environment.

**Note:** This tutorial uses Omniverse Kit's Python API, which is accessible in Omniverse Code or Omniverse Isaac Sim.

## Setting Up the Environment

First, we'll set up the scripting environment in Omniverse.

### Steps:

1. **Launch Omniverse Code or Omniverse Isaac Sim.**
2. **Open the Script Editor:** Go to `Window` > `Script Editor`.
3. **Set Up the Environment:** We'll import necessary modules and set up the stage.

In [ ]:
# Import necessary modules
from omni.isaac.kit import SimulationApp
simulation_app = SimulationApp()

from pxr import Usd, UsdGeom, Gf, Sdf
import omni.usd

# Create a new stage
stage = omni.usd.get_context().get_stage()
if not stage:
    stage = Usd.Stage.CreateInMemory()
    omni.usd.get_context().set_stage(stage)

# Set the stage's up axis and meters per unit
UsdGeom.SetStageUpAxis(stage, UsdGeom.Tokens.y)
stage.SetMetersPerUnit(1.0)

## Generating Uneven Terrain

We'll create a terrain mesh using a plane and apply a displacement map generated via a procedural noise function.

### Steps:

1. **Create a Plane Mesh.**
2. **Generate Height Data using Perlin Noise.**
3. **Apply the Height Data to the Plane's Vertices.**

In [ ]:
# Import noise library
!pip install noise
from noise import pnoise2

# Parameters for the terrain
terrain_width = 100  # meters
terrain_depth = 100  # meters
resolution = 1  # meters between vertices

# Calculate the number of vertices
num_vertices_x = int(terrain_width / resolution)
num_vertices_z = int(terrain_depth / resolution)

# Create lists to store vertex positions and indices
vertex_positions = []
indices = []

# Generate vertex positions with noise
scale = 0.1  # Controls the zoom of the noise
octaves = 6
persistence = 0.5
lacunarity = 2.0

for z in range(num_vertices_z):
    for x in range(num_vertices_x):
        y = pnoise2(x * scale, 
                    z * scale, 
                    octaves=octaves, 
                    persistence=persistence, 
                    lacunarity=lacunarity, 
                    repeatx=1024, 
                    repeaty=1024, 
                    base=0)
        y *= 10  # Amplify the height
        vertex_positions.append(Gf.Vec3f(x * resolution, y, z * resolution))

# Generate indices for triangle faces
for z in range(num_vertices_z - 1):
    for x in range(num_vertices_x - 1):
        i0 = x + z * num_vertices_x
        i1 = i0 + 1
        i2 = i0 + num_vertices_x
        i3 = i2 + 1

        # First triangle
        indices.extend([i0, i1, i2])
        # Second triangle
        indices.extend([i1, i3, i2])

# Create the mesh in the USD stage
terrain_path = Sdf.Path("/World/Terrain")
terrain_mesh = UsdGeom.Mesh.Define(stage, terrain_path)

# Set the vertex positions
terrain_mesh.GetPointsAttr().Set(vertex_positions)

# Set the face vertex indices and counts
terrain_mesh.GetFaceVertexIndicesAttr().Set(indices)
terrain_mesh.GetFaceVertexCountsAttr().Set([3] * (len(indices) // 3))

# Add the terrain to the stage
stage.GetRootLayer().Save()

**Explanation:**

- **Perlin Noise (`pnoise2`)** is used to generate realistic terrain elevations.
- **Vertex Positions** are calculated based on the noise values.
- **Indices** define how vertices are connected to form triangles.

## Applying Textures and Materials

To make the terrain look realistic, we'll apply textures and materials.

### Steps:

1. **Create a Material.**
2. **Assign the Material to the Terrain Mesh.**
3. **Apply a Texture Map.**

In [ ]:
# Create a material
from omni.usd import get_material_prim
import omni.kit.commands

material_path = "/World/Terrain/Material"

# Create a new material
omni.kit.commands.execute('CreateAndBindMdlMaterialFromLibrary', 
    mdl_name='OmniPBR.mdl', 
    mtl_name='OmniPBR', 
    prim_path=material_path,
    bind_selected_prims=False)

# Assign the material to the terrain
terrain_mesh.GetPrim().GetRelationship('material:binding').SetTargets([Sdf.Path(material_path)])

# Set up the texture
material_prim = stage.GetPrimAtPath(material_path)
shader = material_prim.GetChild('Shader')

# Assuming you have a texture file at the specified path
texture_file = "omniverse://localhost/Materials/Textures/grass.png"

# Set the texture
shader.GetAttribute('inputs:diffuse_texture').Set(texture_file)

# Save the stage
stage.GetRootLayer().Save()

**Explanation:**

- We use **OmniPBR** material for realistic rendering.
- The **diffuse_texture** input is set to a grass texture.
- The texture file should be accessible; adjust the path accordingly.

## Simulating the Terrain

Now that the terrain is created and textured, we'll set up the simulation environment.

### Steps:

1. **Add Lighting and Sky.**
2. **Set up a Camera.**
3. **Run the Simulation.**

In [ ]:
# Add a distant light
distant_light = UsdGeom.DistantLight.Define(stage, "/World/DistantLight")
distant_light.CreateIntensityAttr(500)

# Add a dome light for sky
dome_light = UsdGeom.DomeLight.Define(stage, "/World/DomeLight")
dome_light.CreateTextureFileAttr("omniverse://localhost/Materials/Skies/ClearSky.exr")

# Add a camera
camera = UsdGeom.Camera.Define(stage, "/World/Camera")
camera.CreateFocalLengthAttr(24)
camera.AddTranslateOp().Set(Gf.Vec3f(50, 50, 150))
camera.AddRotateYOp().Set(180)
camera.AddRotateXOp().Set(-45)

# Set the default prim to the camera
stage.SetDefaultPrim(camera.GetPrim())

# Save the stage
stage.GetRootLayer().Save()

# Run the simulation
simulation_app.update()
simulation_app.update()

# Stop the simulation
simulation_app.stop()
simulation_app.close()

**Explanation:**

- **Distant Light** simulates sunlight.
- **Dome Light** provides ambient lighting and sky texture.
- **Camera** is positioned to view the terrain.
- **Simulation Application** runs the scene.

## Conclusion

You've successfully created an uneven outdoor terrain in NVIDIA Omniverse using Python scripting. This terrain can now be used for simulations, visualizations, or as a starting point for more complex environments.

### Next Steps

- Experiment with different noise parameters to create varied terrains.
- Add more environmental details like trees, rocks, or water bodies.
- Integrate physics simulations or robotic agents to interact with the terrain.

### Additional Resources:

- NVIDIA Omniverse Documentation: https://docs.omniverse.nvidia.com/
- Omniverse Code Tutorials: https://docs.omniverse.nvidia.com/app_code/app_code/tutorial.html
- Perlin Noise Reference: https://github.com/caseman/noise

---

**Happy Modeling!**